# Creating classes in Python

# Attribute vs property and class containers

Germain Salvato Vallverdu


## Without property

In [1]:
class Leaf:
    """ This class represents a leaf's tree """
    
    def __init__(self, width: float, length: float, color: str = "green"):
        
        self.width = width
        self.length = length
        self.color = color
        
    def get_area(self):
        return self.width * self.length

In [2]:
l = Leaf(4, 12)

In [4]:
l

In [5]:
l.width, l.length, l.color

(4, 12, 'green')

In [6]:
l.get_area()

48

By default there is no control on attribute. You can thus change their value.

In [7]:
# not any control
l.width = -5
l.get_area()

-60

All python object are fully dynamic. Attribute are thus created on the fly even
on instances.

In [11]:
# really no control
l.weight = 18
vars(l)

{'width': -5, 'length': 12, 'color': 'green', 'weight': 18}

## Define properties

In [14]:
class Leaf:
    """ This class represents a leaf's tree """
    
    def __init__(self, width: float, length: float, color: str = "green"):
        
        self.width = width
        self.length = length
        self.color = color
        
        # for the sake of efficiency it is better to compute it one times in
        # the init. If you don't, you will compute it each time you use the
        # property
        self._area = self.width * self.length
   
    @property
    def area(self):
        """ Area of the leaf """
        return self._area

In [15]:
l = Leaf(4, 12)

`area` is now a property and no more a function/method.

In [16]:
l.area

48

You are still able to set the `width` or other attributres but not the `area` which is a property.

This leads to strange or erroneous behavior.

In [23]:
l.width = -5
l.area

48

In [29]:
try:
    l.area = -5
except AttributeError as error:
    print("#ERROR#", error)

can't set attribute


## Define more properties

We assume the following:

* area is a property computed from init data
* width and length cannot be changed after the instance is created

In [31]:
class Leaf:
    """ This class represents a leaf's tree """
    
    def __init__(self, width: float, length: float, color: str = "green"):
        
        self._width = width
        self._length = length
        self.color = color
        
        # for the sake of efficiency it is better to compute it one times in
        # the init. If you don't, you will compute it each time you use the
        # property
        self._area = self.width * self.length
   
    @property
    def area(self):
        """ Area of the leaf """
        return self._area
    
    @property
    def width(self):
        """ leaf's width """
        return self._width

    @property
    def length(self):
        """ leaf's width """
        return self._length

In [32]:
l = Leaf(4, 12)

In [33]:
try:
    l.width = -5
except AttributeError as error:
    print("#ERROR#", error)

can't set attribute


In [40]:
try:
    l.length = -5
except AttributeError as error:
    print("#ERROR#", error)

can't set attribute


But it is still possible to modify hidden attributes

In [41]:
print(vars(l))

{'_width': 4, '_length': 12, 'color': 'green', '_area': 48}


In [46]:
l._width = - 5
print(vars(l))
print("area =", l.area)

{'_width': -5, '_length': 12, 'color': 'green', '_area': 48}
area = 48


## Define properties and setters

We assume the following:

* area is a property computed from instance data
* width and length can be update but with control

In [49]:
class Leaf:
    """ This class represents a leaf's tree """
    
    def __init__(self, width: float, length: float, color: str = "green"):
        
        self._width = width
        self._length = length
        self.color = color
        
        # for the sake of efficiency it is better to compute it one times in
        # the init. If you don't, you will compute it each time you use the
        # property
        self._area = self.width * self.length
   
    @property
    def area(self):
        """ Area of the leaf """
        return self._area
    
    @property
    def width(self):
        """ leaf's width """
        return self._width
    
    @width.setter
    def width(self, val):
        """ check width and upadte self """
        if val > 0:
            self._width = val
            self._area = self._width * self._length
        else:
            raise ValueError(f"Wrong width values: '{val}'")

    # other option to define property
    def get_length(self):
        """ leaf's width """
        return self._length
    
    def set_length(self, val):
        """ check width and upadte self """
        if val > 0:
            self._length = val
            self._area = self._width * self._length
        else:
            raise ValueError(f"Wrong width values: '{val}'")
    
    length = property(get_length, set_length)

In [57]:
l = Leaf(4, 12)

In [59]:
print(vars(l), "    area =", l.area)
l.length = 24
print(vars(l), "    area =", l.area)
l.width = 5
print(vars(l), "    area =", l.area)
try:
    l.width = -1
except ValueError as error:
    print("#ERROR#", error)

{'_width': 5, '_length': 24, 'color': 'green', '_area': 120}     area = 120
{'_width': 5, '_length': 24, 'color': 'green', '_area': 120}     area = 120
{'_width': 5, '_length': 24, 'color': 'green', '_area': 120}     area = 120
#ERROR# Wrong width values: '-1'


## Fast class writing 

There are several class factories in python that write some parts of the code for you. For
example the special methods `__init__` or `__repr__` or others.

### Use NamedTuple

`namedtuple` is a very fast way to get a datacontainer with imutable fields (as tuples).

In [71]:
from collections import namedtuple

In [102]:
Leaf = namedtuple("Leaf", ["width", "length", "color"], defaults=["green"])

In [103]:
l = Leaf(4, 12)
l

Leaf(width=4, length=12, color='green')

In [104]:
l2 = Leaf(4, 12)
l == l2  # this comparison was not available in previous implementation.

True

In [105]:
l.width, l.length

(4, 12)

In [106]:
for item in l:
    print(item)

4
12
green


In [107]:
for i in range(3):
    print(l[i])

4
12
green


NamedTuple are immutable object. You cannot set attributes.

In [109]:
try:
    l.length = 5
except AttributeError as error:
    print(error)

can't set attribute


In [111]:
try:
    l.area = l.length * l.width
except AttributeError as error:
    print(error)

'Leaf' object has no attribute 'area'


NamedTuple is very fast but very basic. For example, here `area` is not available. You can however extend it:

In [112]:
class Leaf(namedtuple(
    "Leaf", ["width", "length", "color"], defaults=["green"])):
    
    @property
    def area(self):
        return self.width * self.length

In [114]:
l = Leaf(4, 12)
l

Leaf(width=4, length=12, color='green')

In [115]:
l.area

48

### Dataclass Example

The aim of dataclass is to be a container. By default, the aim is not to protect
data. In the following attributed are thus editable.

In [88]:
from dataclasses import dataclass, field

In [116]:
@dataclass
class Leaf:
    """ This class represents a leaf's tree """
    
    width: float
    length: float
    color: str = "green"
    
    area: float = field(init=False)
    
    def __post_init__(self):
        self.area = self.width * self.length

In [122]:
l = Leaf(4, 12)
l

Leaf(width=4, length=12, color='green', area=48)

In [123]:
l.area

48

In [124]:
vars(l)

{'width': 4, 'length': 12, 'color': 'green', 'area': 48}

In [125]:
# you loose the control on attributed relationship
l.length = 2
vars(l)

{'width': 4, 'length': 2, 'color': 'green', 'area': 48}

### Kind of Dataclass with validation

Fine tuning of container classes can be achieve using pydantic `BaseModel`.

In [143]:
from enum import Enum
from pydantic import BaseModel, NonNegativeFloat, ValidationError, validator

In [156]:
class ColorChoice(str, Enum):
    green = "green"
    very_green = "very_green"
    so_green = "so_green"

class Leaf(BaseModel):
    width: NonNegativeFloat
    length: NonNegativeFloat
    color: ColorChoice = ColorChoice.green
    
    @validator("length")
    def length_lt_width_validation(cls, v, values):
        if v < values["width"]:
            raise ValueError(f"Length must be larger than width.")
        else:
            return v

    @property
    def area(self):
        return self.width * self.length

In [148]:
l = Leaf(width=3, length=8)
l

Leaf(width=3.0, length=8.0, color=<ColorChoice.green: 'green'>)

In [149]:
l.area

24.0

In [138]:
try:
    Leaf(width=-2, length=2)
except ValidationError as error:
    print(error)

1 validation error for Leaf
width
  ensure this value is greater than or equal to 0 (type=value_error.number.not_ge; limit_value=0)


In [153]:
try:
    Leaf(width=12, length=2)
except ValueError as error:
    print(error)

1 validation error for Leaf
length
  Length must be larger than width. (type=value_error)


In [157]:
l.dict()

{'width': 3.0, 'length': 8.0, 'color': <ColorChoice.green: 'green'>}

In [158]:
l.json()

'{"width": 3.0, "length": 8.0, "color": "green"}'